From Wageningen B-series open water propeller performance charts (https://doi.org/10.5281/zenodo.8352831):

As early as 1883, R.E. Froude, after extensive experiments with model propellers behind ship models, and model propellers separate from the ship, found that a propeller behind a ship, rotating at a rotational velocityn , behaves like a propeller separate from the ship, operating at the same rotational velocityn , at a reduced advance velocity $V_0 < V$ , called ‘effective advance velocity’. He then introduced the ‘effective wake fraction’w and the ‘relative rotative efficiency $\eta_R$ as the factors relating the two conditions, according to the following equations (thrust equalization method is assumed, Taylor’s definition of wake fraction $W$ is used in (7)):

$$
\begin{aligned}
T_0 &= T (5) \\
Q_0 &= Q \eta_R (6) \\
V_0 &= V (1 - W) (7)
\end{aligned}
$$

where $T_0$, $V_0$, $Q_0$ denote the propeller thrust, effective speed of advance and torque of the propeller operating separate from the ship and, $T$, $V$, $Q$ the propeller thrust,
ship’s speed and torque of the propeller operating behind ship.

We use the following notation for the dimensionless variables for the propeller operation separate from the ship (open water or free running conditions):

$$
\begin{aligned}
J_0 &= V_0 / (n D) : advance coefficient, open-water (8) \\
k_{t0} &= T_0 / (\rho n^2 D^4) : thrust coefficient, open-water (9) \\
k_{q0} &= Q_0 / (\rho n^2 D^5): torque coefficient, open-water (10) \\
\eta_0 &= T_0 V_0 / (2 \pi n Q_0) = J_0 k_{t0} / (2 \pi k_{q0}): propeller efficiency, open-water (11)
\end{aligned}
$$

where $n$ is the propeller rotational velocity and $D$ the propeller diameter.

We also use the following notation for the dimensionless variables for the propeller operation behind ship (self-propulsion conditions):

$$
\begin{aligned}
J &= V / (n D) : advance coefficient, self-propulsion (12) \\
k_t &= T / (\rho n^2 D^4) : thrust coefficient, self-propulsion (13) \\
k_q &= Q / (\rho n^2 D^5) = DHP / (2 \pi p n^3 D^5): torque coefficient, self-propulsion (14) \\
\eta &= T V / (2 \pi n Q) = J k_{t0} / (2 \pi k_q): propeller efficiency, self-propulsion (15)
\end{aligned}
$$

where $DHP = 2 \pi n Q$ the delivered horsepower of the propeller operating behind ship.

Due to equations (5) to (7), dimensionless variables for propeller operation separate from ship and behind ship, are related as follows:

$$
\begin{aligned}
J_0 &= J (1 - w) <-> J = J_0 / (1 - w) \\
k_{t0} &= k_t \\
k_{q0} &= \eta_R k_q <-> k_q = k_{q0} / \eta_R \\
\eta_0 &= \eta (1 - w) / \eta_R <-> \eta = \eta_0 \eta_R / (1 - w)
\end{aligned}
$$


In [1]:
import importlib.util
import json
import numpy as np
import pandas as pd
import sys
from pathlib import Path

sys.path.append(".")
sys.path.append("../hull")
sys.path.append("../../..")

from helper import (
    estimate_initial_values,
    load_boat_data,
)
from utils.optimization import load_model_params_from_json

# Load ESC and Motor models from sibling folders
esc_spec = importlib.util.spec_from_file_location("esc_model", Path("../esc/model.py"))
esc_model = importlib.util.module_from_spec(esc_spec)
esc_spec.loader.exec_module(esc_model)
ESC = esc_model.ESC

motor_spec = importlib.util.spec_from_file_location(
    "motor_model", Path("../motor/model.py")
)
motor_model = importlib.util.module_from_spec(motor_spec)
motor_spec.loader.exec_module(motor_model)
Motor = motor_model.SteadyStateMotor

hull_spec = importlib.util.spec_from_file_location(
    "hull_model", Path("../hull/hydrostatic_hull.py")
)
hull_model = importlib.util.module_from_spec(hull_spec)
hull_spec.loader.exec_module(hull_model)
HydrostaticHull = hull_model.HydrostaticHull

In [2]:
# Load blade geometry results from image (exported by blade_area_from_image.ipynb)
with open("propeller_blade_geometry.json", "r", encoding="utf-8") as f:
    blade_geom_json = json.load(f)

blade_geom = {
    "prop_D": blade_geom_json["prop_diameter_cm"] / 100,
    "prop_AEA0": blade_geom_json["AEA0"],
    "prop_c_07": blade_geom_json["chord_cm"] / 100,
}

In [3]:
hull_input_params = load_model_params_from_json(
    "../../../models/2020/hull/data/hull_input_params.json"
)

hull = HydrostaticHull(
    cog_x=hull_input_params["cog_x"],
    disp_mass=hull_input_params["disp_mass"],
    total_area=hull_input_params["total_area"],
)
hull_params = hull.get_params()

hull_LWL = hull_params["LWL"]
hull_BWL = hull_params["BWL"]
hull_M = hull_params["disp_mass"]
hull_S_water = hull_params["wet_surface_area_interp"]
hull_S_air = np.abs(hull_params["total_area"] - hull_params["wet_surface_area_interp"])

In [4]:
# Load model parameters
esc_params = load_model_params_from_json("../../../models/2020/esc/mam17_params.json")
motor_params = load_model_params_from_json(
    "../../../models/2020/motor/me0909_free_rotor_step_params.json"
)

# Load boat dataset and select operating point near max input power
df = load_boat_data("../../../models/2020/boat_data_1s.csv")

# Getting system operating point:
train_section = dict(
    start=pd.Timestamp("2020-02-02 12:58:50"), end=pd.Timestamp("2020-02-02 13:03:30")
)
df = df.loc[(df.index >= train_section["start"]) & (df.index <= train_section["end"])][
    130:
]
idx_max = int(np.nanargmax(df["pilot_d"]))
op = df.iloc[idx_max + 2 : idx_max + 40].mean(numeric_only=True)

display(op)

batt_v = op["batt_v"]
pilot_d = op["pilot_d"]
motor_w = op["motor_w"]
esc_i_in = op["esc_i_in"]
hull_u = op["hull_u"]

batt_v          32.338314
pilot_d          1.000000
motor_w        282.798416
esc_p_in      4078.474490
esc_i_in       126.186759
t           338762.500000
hull_u           4.881316
dtype: float64

In [5]:
import numpy as np
from scipy.optimize import root  # or fsolve


def solve_op_q_load(
    *,
    batt_v_meas: float,
    pilot_d: float,
    esc_i_in_meas: float,
    motor_params: dict,
    esc_params: dict,
    motor_w_guess: float,
):
    # Initial guesses
    w0 = float(motor_w_guess)
    # crude q guess from electrical power / omega (assume some efficiency)
    # avoid divide-by-zero if w0 ~ 0
    q0 = 0.7 * (batt_v_meas * max(esc_i_in_meas, 0.0)) / max(w0, 1e-3)

    def residuals(x):
        motor_w, motor_q_load = x

        # 1) ESC -> motor voltage guess (start with previous iterate’s implied vout)
        # We'll compute esc_v_out_hat using motor_i_hat, but motor_i_hat needs motor_v.
        # So do a tiny fixed-point inside: start with v = pilot_d*batt_v then update once.
        motor_v = pilot_d * batt_v_meas

        # One (or a few) refinement iterations usually helps convergence
        for _ in range(2):
            motor_inputs = np.array([motor_v, motor_q_load], dtype=float)
            motor_i_hat, motor_w_hat = Motor._outputs(
                None, None, motor_inputs, motor_params
            )

            esc_inputs = np.array([batt_v_meas, motor_i_hat, pilot_d], dtype=float)
            esc_v_out_hat, esc_i_in_hat = ESC._outputs(
                None, None, esc_inputs, esc_params
            )

            motor_v = esc_v_out_hat  # enforce coupling for next refinement

        # Two equations:
        r1 = motor_w_hat - motor_w
        r2 = esc_i_in_hat - esc_i_in_meas
        return np.array([r1, r2], dtype=float)

    sol = root(
        residuals,
        x0=np.array([w0, q0], dtype=float),
        method="hybr",  # same family as fsolve
        tol=1e-10,
    )
    if not sol.success:
        raise RuntimeError(f"root() failed: {sol.message}")

    motor_w_sol, motor_q_load_sol = sol.x

    # Recompute “final” coupled outputs at the solution
    motor_v = pilot_d * batt_v_meas
    for _ in range(3):
        motor_inputs = np.array([motor_v, motor_q_load_sol], dtype=float)
        motor_i_hat, motor_w_hat = Motor._outputs(
            None, None, motor_inputs, motor_params
        )

        esc_inputs = np.array([batt_v_meas, motor_i_hat, pilot_d], dtype=float)
        esc_v_out_hat, esc_i_in_hat = ESC._outputs(None, None, esc_inputs, esc_params)
        motor_v = esc_v_out_hat

    # Compute power and efficiencies
    motor_p_in = motor_i_hat * motor_v
    motor_p_out = motor_q_load_sol * motor_w_hat
    motor_eta = motor_p_out / motor_p_in if motor_p_in != 0 else np.nan
    esc_i_out = motor_i_hat
    esc_p_out = esc_v_out_hat * esc_i_out
    esc_v_in = batt_v
    esc_p_in = esc_v_in * esc_i_in
    esc_eta = esc_p_out / esc_p_in if esc_p_in != 0 else np.nan

    return {
        "motor_w": float(motor_w_sol),
        "motor_q_load": float(motor_q_load_sol),
        "motor_v": float(motor_v),
        "motor_i": float(motor_i_hat),
        "motor_w": float(motor_w_hat),
        "motor_p_out": float(motor_p_out),
        "motor_p_in": float(motor_p_in),
        "motor_eta": float(motor_eta),
        "esc_v_out": float(esc_v_out_hat),
        "esc_i_in": float(esc_i_in_hat),
        "esc_p_out": float(esc_p_out),
        "esc_p_in": float(esc_p_in),
        "esc_eta": float(esc_eta),
        "esc_d": float(pilot_d),
        "converged": bool(sol.success),
        "residuals": sol.fun.astype(float),
    }


motor_esc_operating_point = solve_op_q_load(
    batt_v_meas=float(op["batt_v"]),
    pilot_d=float(op["pilot_d"]),
    esc_i_in_meas=float(op["esc_i_in"]),
    motor_params=motor_params,
    esc_params=esc_params,
    motor_w_guess=float(op["motor_w"]),
)
motor_esc_operating_point

{'motor_w': 289.6992393996395,
 'motor_q_load': 12.50893515724535,
 'motor_v': 32.33831393975976,
 'motor_i': 122.15983072126319,
 'motor_p_out': 3623.829000753388,
 'motor_p_in': 3950.4429566921176,
 'motor_eta': 0.9173221941135892,
 'esc_v_out': 32.33831393975976,
 'esc_i_in': 126.18675911851751,
 'esc_p_out': 3950.4429566921176,
 'esc_p_in': 4080.667031415461,
 'esc_eta': 0.968087551931878,
 'esc_d': 1.0,
 'converged': True,
 'residuals': array([0.00000000e+00, 1.42108547e-14])}

In [6]:
# Estimate propeller and chain performance
in_to_m = 0.0254
radps_to_rpm = 60 / (2 * np.pi)

vals = estimate_initial_values(
    motor_rpm=motor_w * radps_to_rpm,
    batt_v=batt_v,
    esc_i_in=motor_esc_operating_point["motor_i"],
    motor_eta=motor_esc_operating_point["motor_eta"],
    esc_eta=motor_esc_operating_point["esc_eta"],
    trans_eta=0.95,
    trans_k=14 / 22,
    hull_u=hull_u,
    hull_L=hull_LWL,
    hull_B=hull_BWL,
    hull_M=hull_M,
    hull_S_water=hull_S_water,
    hull_S_air=hull_S_air,
    hull_C_TB=0.385,
    prop_D=blade_geom["prop_D"],
    prop_Z=3,
    prop_hub_D=0.06,
    prop_AEA0=blade_geom["prop_AEA0"],
    prop_PD=10.5 / 9,
    prop_total_m=0.66,
    prop_blade_thickness=0.005,
    prop_mat_rho=2700.0,
    prop_c_07=blade_geom["prop_c_07"],
    water_rho=1023,  # Estimated from public ocean dataset
    air_rho=1.1839,  # Estimated from public ocean dataset
    water_mu=0.0009268,  # Estimated from public ocean dataset
)
vals

{'motor_rpm': 2700.525944653284,
 'motor_n': 45.0087657442214,
 'motor_w': 282.79841561837975,
 'motor_v': 32.33831393975976,
 'motor_i': 122.15983072126319,
 'motor_eta': 0.9173221941135892,
 'esc_eta': 0.968087551931878,
 'trans_eta': 0.95,
 'trans_k': 0.6363636363636364,
 'esc_P_in': 3950.4429566921176,
 'esc_P_out': 3824.3746509906023,
 'motor_P_shaft': 3508.183745959091,
 'trans_P_out': 3332.7745586611363,
 'system_eta_elec_to_prop': 0.966034368101119,
 'system_eta_available_to_prop': 1.1450710506265627,
 'hull_u': 4.881316409302568,
 'hull_L': 5.202762254164356,
 'hull_B': 0.9634856741038357,
 'hull_M': 293.7,
 'hull_C_TB': 0.385,
 'hull_S_air': 4.188174709710495,
 'hull_S_water': 4.050226176680205,
 'hull_C_T': 0.00830774302162304,
 'hull_T': 0.14876074371822862,
 'hull_Fn': 0.6832591646582801,
 'hull_W': 0.14250000000000002,
 'hull_T_ded': 0.0855,
 'hull_k_R': 0.6,
 'hull_eta_H': 1.0664723032069972,
 'prop_rpm': 1718.516510233908,
 'prop_n': 28.6419418372318,
 'prop_w': 179.962

In [7]:
from utils.optimization import save_model_params_to_json

save_model_params_to_json("data/propulsive_initial_params.json", vals)